## 自动微分
训练神经网络时候最常用的算法是 反向传播

算法根据损失函数对于给定参数的梯度来调整参数（模型的权重）

mindspore 计算一阶导数的方法是

 `mindspore.ops.GradOperation(get_all=False, get_by_list=False, sens_param=False)`
 
get_all=False 只会对第一个输入求导 True对所有输入求导
get_list=False 不会对权重求导 True 对权重求导
sens_param 对网络的输出值做缩放以改变最终梯度 

对 MatMul 算子的求导做深入分析

导包

In [13]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor
from mindspore import ParameterTuple, Parameter
from mindspore import dtype as mstype

### 对输入求一阶导
首先需要定义一个需要求导的网络，以一个MatMul算子构成的网络f(x,y) = z*x*y为例
网络如下

In [14]:
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.matmul = ops.MatMul()
        # 在这里就可以指定使用的类型是mindspore.float32或者别的mstype类
        self.z = Parameter(Tensor(np.array([1.0]),mstype.float32),name = 'z') 
        # print(type(self.z))
    
    def construct(self,x,y):
        x = x*self.z
        out = self.matmul(x,y)
        return out

##### 定义求导网络

__init__定义需要求到的网络self.net 和 ops.GradOperation操作

construct 对self.net进行求导

In [21]:
class GradNetWrtY(nn.Cell):
    def __init__(self,net):
        super(GradNetWrtY,self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()
        self.grad_op1 = ops.GradOperation(get_all=True)

    def construct(self,x,y):
        gradient_function = self.grad_op1(self.net)
        return gradient_function(x,y)

定义输入并且打印输出

In [22]:
x = Tensor([[0.8,0.6,0.2],[1.8,1.3,1.1]],dtype=mstype.float32)
y = Tensor([[0.11,3.3,1.1],[1.1,0.2,1.4],[1.1,2.2,0.3]],dtype=mstype.float32)
output = GradNetWrtY(Net()).construct(x,y)
print(output)

<class 'mindspore.common.parameter.ParameterTensor'>
(Tensor(shape=[2, 3], dtype=Float32, value=
[[4.50999975e+000, 2.70000005e+000, 3.60000014e+000],
 [4.50999975e+000, 2.70000005e+000, 3.60000014e+000]]), Tensor(shape=[3, 3], dtype=Float32, value=
[[2.59999990e+000, 2.59999990e+000, 2.59999990e+000],
 [1.89999998e+000, 1.89999998e+000, 1.89999998e+000],
 [1.30000007e+000, 1.30000007e+000, 1.30000007e+000]]))


### 对权重求一阶导
将 ops.GradOperation 中的 get_by_list 设置为 True

In [25]:
class GradNetWrtX(nn.Cell):
    def __init__(self,net):
        super(GradNetWrtX,self).__init__()
        self.net = net
        self.param = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self,x,y):
        gradient_function = self.grad_op(self.net,self.param)
        return gradient_function(x,y)
    
output = GradNetWrtX(Net()).construct(x,y)
print(output)


<class 'mindspore.common.parameter.ParameterTensor'>
(Tensor(shape=[1], dtype=Float32, value= [2.15359993e+001]),)


如果需要某些权重不求导？

在定义求导网络的时候，对应的权重中 `requires_grad`设置为False
```python
self.z = Parameter(Tensor(np.array([1.0])),mstype.float32,name = 'z',required_grad = False)
```

## 梯度值缩放
可以通过`sens_param` 参数对网络的输出值做缩放以改变最终梯度

首先 `ops.GradOperation`的`sens_param`设置为True

缩放指数 `self.grad_wrt_output` 可以这样记录
```python
    self.grad_wrt_output = Tensor([s1,s2,s3],[s4,s5,s6])
```
则GradNetWrtZ的结构为

In [27]:
class GradNetWrtZ(nn.Cell):
    def __init__(self,net):
        super(GradNetWrtZ,self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation(sens_param=True)
        self.grad_wrt_output = Tensor(np.array(([[2,2,2],[1,1,1]])),dtype=mstype.float32)
        # 缩放指数按照上述设置，最后的结果中 第一列都是第二列的两倍 这就是缩放指数的作用

    def construct(self,x,y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x,y,self.grad_wrt_output)
    
output = GradNetWrtZ(Net()).construct(x,y)
print(output)

<class 'mindspore.common.parameter.ParameterTensor'>
[[9.0199995 5.4       7.2000003]
 [4.5099998 2.7       3.6000001]]


## 停止计算梯度
可以使用stop_gradient来禁止网络内算子对梯度的影响

In [32]:
from mindspore.ops.functional import stop_gradient
class Net1(nn.Cell):
    def __init__(self):
        super(Net1,self).__init__()
        self.matmul = ops.MatMul()
    def construct(self,x,y):
        out1 = self.matmul(x,y)
        out2 = self.matmul(x,y)
        # out2 = stop_gradient(out2) 
        out = out1+out2
        return out       
    
class GradNetWrtO(nn.Cell):
    def __init__(self,net):
        super(GradNetWrtO,self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()
    
    def construct(self,x,y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x,y)
    
output = GradNetWrtO(Net1())(x,y)
print(output)
        
        

[[9.0199995 5.4       7.2000003]
 [9.0199995 5.4       7.2000003]]
